<a href="https://colab.research.google.com/github/sum-coderepo/02456-deep-learning-with-PyTorch/blob/master/Seq2Seq/SeqToSeq_pytorchLSTMEnd2ENd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%md
End to end SeqToSeq model from reading the text corpus to training and prediction,


In [ ]:
!pip install torchtext 
!python -m spacy download de_core_news_sm
!python3 -m spacy download en_core_web_sm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time


In [ ]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torchtext.utils import download_from_url, extract_archive
import io

url_base = 'https://raw.githubusercontent.com/multi30k/dataset/master/data/task1/raw/'
train_urls = ('train.de.gz', 'train.en.gz')
val_urls = ('val.de.gz', 'val.en.gz')
test_urls = ('test_2016_flickr.de.gz', 'test_2016_flickr.en.gz')

train_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in train_urls]
val_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in val_urls]
test_filepaths = [extract_archive(download_from_url(url_base + url))[0] for url in test_urls]

In [ ]:
de_tokenizer = get_tokenizer('spacy', language='de')
en_tokenizer = get_tokenizer('spacy', language='en')

/databricks/python/lib/python3.7/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
 warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')
/databricks/python/lib/python3.7/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
 warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')

In [ ]:
def build_vocab(filepath, tokenizer):
  counter = Counter()
  with io.open(filepath, encoding="utf8") as f:
    for string_ in f:
      counter.update(tokenizer(string_))
  return Vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

In [ ]:
de_vocab = build_vocab(train_filepaths[0], de_tokenizer)
en_vocab = build_vocab(train_filepaths[1], en_tokenizer)

In [ ]:
en_vocab.freqs, en_vocab.stoi

Out[7]: (Counter({'Two': 3116,
 'young': 2074,
 ',': 3963,
 'White': 23,
 'males': 39,
 'are': 3716,
 'outside': 870,
 'near': 569,
 'many': 120,
 'bushes': 19,
 '.': 27623,
 '\n': 29000,
 'Several': 280,
 'men': 1612,
 'in': 14847,
 'hard': 120,
 'hats': 146,
 'operating': 22,
 'a': 31707,
 'giant': 43,
 'pulley': 3,
 'system': 6,
 'A': 17458,
 'little': 843,
 'girl': 1667,
 'climbing': 193,
 'into': 711,
 'wooden': 174,
 'playhouse': 3,
 'man': 7359,
 'blue': 1871,
 'shirt': 2323,
 'is': 7524,
 'standing': 1612,
 'on': 8019,
 'ladder': 60,
 'cleaning': 60,
 'window': 183,
 'at': 2908,
 'the': 9923,
 'stove': 23,
 'preparing': 104,
 'food': 352,
 'green': 920,
 'holds': 322,
 'guitar': 365,
 'while': 1932,
 'other': 660,
 'observes': 13,
 'his': 1969,
 'smiling': 344,
 'stuffed': 32,
 'lion': 7,
 'trendy': 2,
 'talking': 382,
 'her': 1263,
 'cellphone': 144,
 'gliding': 5,
 'slowly': 3,
 'down': 1402,
 'street': 1432,
 'woman': 3816,
 'with': 6175,
 'large': 825,
 'purse': 52,
 'walking': 1376,
 'by': 1046,
 'gate': 27,
 'Boys': 12,
 'dancing': 183,
 'poles': 27,
 'middle': 256,
 'of': 6871,
 'night': 137,
 'ballet': 11,
 'class': 55,
 'five': 46,
 'girls': 424,
 'jumping': 491,
 'sequence': 2,
 'Four': 314,
 'guys': 101,
 'three': 202,
 'wearing': 2622,
 'one': 918,
 'not': 30,
 'top': 414,
 'staircase': 30,
 'black': 1985,
 'dog': 1651,
 'and': 7378,
 'spotted': 13,
 'fighting': 51,
 'neon': 22,
 'orange': 520,
 'uniform': 192,
 'driving': 68,
 'tractor': 25,
 'women': 919,
 'wait': 52,
 'city': 432,
 'lady': 362,
 'glasses': 306,
 'sprinkling': 3,
 'powdered': 2,
 'sugar': 1,
 'bundt': 1,
 'cake': 56,
 'sitting': 1740,
 'front': 1273,
 'painted': 65,
 'rainbow': 15,
 'lays': 31,
 'bench': 310,
 'to': 3127,
 'which': 83,
 'white': 2215,
 'also': 20,
 'tied': 16,
 'Five': 127,
 'people': 2546,
 'circle': 33,
 'instruments': 86,
 'bunch': 60,
 'elderly': 180,
 'play': 363,
 'their': 723,
 'clarinets': 1,
 'together': 361,
 'as': 845,
 'they': 170,
 'read': 19,
 'off': 405,
 'sheet': 28,
 'music': 118,
 'structure': 66,
 'has': 339,
 'broken': 16,
 'laying': 149,
 'roadway': 4,
 'crowd': 507,
 'stand': 447,
 'entrance': 19,
 'Metro': 1,
 'station': 68,
 'getting': 167,
 'tattoo': 37,
 'back': 244,
 'children': 668,
 'sit': 301,
 'small': 638,
 'seesaw': 5,
 'sand': 214,
 'reflective': 17,
 'vest': 118,
 'hat': 724,
 'flag': 84,
 'road': 312,
 'person': 689,
 'dressed': 640,
 'coat': 228,
 'busy': 160,
 'sidewalk': 532,
 'studying': 13,
 'painting': 124,
 'scene': 59,
 'pants': 306,
 'The': 1032,
 'child': 869,
 'climbs': 52,
 'red': 1743,
 'ropes': 21,
 'playground': 83,
 'You': 4,
 'know': 4,
 'i': 7,
 'am': 4,
 'looking': 890,
 'like': 117,
 'Justin': 1,
 'Bieber': 1,
 'yellow': 777,
 'jacket': 564,
 'gazing': 9,
 'something': 380,
 'urinal': 2,
 'coffee': 63,
 'cup': 66,
 'multicolored': 41,
 'sky': 75,
 'background': 460,
 'old': 276,
 'having': 173,
 'beer': 75,
 'alone': 44,
 'trained': 4,
 'police': 95,
 'sits': 492,
 'next': 663,
 'handler': 2,
 'van': 38,
 'riding': 583,
 'bike': 445,
 'snowy': 120,
 'uniformly': 1,
 'shirts': 179,
 'tie': 53,
 'slacks': 7,
 'converse': 13,
 'an': 1943,
 'open': 132,
 'backwards': 24,
 'works': 76,
 'machinery': 20,
 'working': 359,
 'factory': 14,
 'setting': 92,
 'packing': 4,
 'jars': 3,
 'candles': 16,
 'boxes': 30,
 'Asian': 326,
 'sweeping': 37,
 'walkway': 49,
 'leans': 45,
 'car': 325,
 'talk': 48,
 'driver': 28,
 'bicycle': 287,
 'looks': 417,
 'toddlers': 10,
 'grass': 468,
 'People': 576,
 'watching': 261,
 'weird': 4,
 'vehicle': 64,
 'plaza': 23,
 'walks': 369,
 'silver': 54,
 'beautiful': 99,
 'bride': 51,
 'new': 38,
 'husband': 6,
 'boy': 1627,
 'playing': 1583,
 'GameCube': 1,
 'McDonald': 6,
 "'s": 444,
 'shakes': 14,
 'edge': 71,
 'beach': 518,
 'ball': 723,
 'group': 1491,
 'barbecue': 20,
 'park': 384,
 'sunglasses': 206,
 'puts': 30,
 'arm': 94,
 'around': 561,
 'blouse': 31,
 'balloon': 58,
 'eating': 236,
 'outdoors': 114,
 'picnic': 32,
 'tables': 46,
 'jump':

In [ ]:
len(en_vocab.itos)

Out[8]: 10838

In [ ]:
def data_process(filepaths):
  raw_de_iter = iter(io.open(filepaths[0], encoding="utf8"))
  raw_en_iter = iter(io.open(filepaths[1], encoding="utf8"))
  data = []
  for (raw_de, raw_en) in zip(raw_de_iter, raw_en_iter):
    de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                            dtype=torch.long)
    en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                            dtype=torch.long)
    data.append((de_tensor_, en_tensor_))
  return data

In [ ]:
train_data = data_process(train_filepaths)
val_data = data_process(val_filepaths)
test_data = data_process(test_filepaths)

In [ ]:
tokens = en_tokenizer("You can now install TorchText using pip!")
tokens

Out[11]: ['You', 'can', 'now', 'install', 'TorchText', 'using', 'pip', '!']

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128
PAD_IDX = de_vocab['<pad>']
BOS_IDX = de_vocab['<bos>']
EOS_IDX = de_vocab['<eos>']

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
  de_batch, en_batch = [], []
  for (de_item, en_item) in data_batch:
    de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
    en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
  de_batch = pad_sequence(de_batch, padding_value=PAD_IDX)
  en_batch = pad_sequence(en_batch, padding_value=PAD_IDX)
  return de_batch, en_batch

In [ ]:
raw_de_iter = iter(io.open(train_filepaths[1], encoding="utf8"))
for i in raw_de_iter:
  print(i) 

Two young, White males are outside near many bushes.

Several men in hard hats are operating a giant pulley system.

A little girl climbing into a wooden playhouse.

A man in a blue shirt is standing on a ladder cleaning a window.

Two men are at the stove preparing food.

A man in green holds a guitar while the other man observes his shirt.

A man is smiling at a stuffed lion

A trendy girl talking on her cellphone while gliding slowly down the street.

A woman with a large purse is walking by a gate.

Boys dancing on poles in the middle of the night.

A ballet class of five girls jumping in sequence.

Four guys three wearing hats one not are jumping at the top of a staircase.

A black dog and a spotted dog are fighting

A man in a neon green and orange uniform is driving on a green tractor.

Several women wait outside in a city.

A lady in a black top with glasses is sprinkling powdered sugar on a bundt cake.

A little girl is sitting in front of a large painted rainbow.

A man lays on the bench to which a white dog is also tied.

Five people are sitting in a circle with instruments.

A bunch of elderly women play their clarinets together as they read off sheet music.

A large structure has broken and is laying in a roadway.

A large crowd of people stand outside in front of the entrance to a Metro station.

A man getting a tattoo on his back.

Two children sit on a small seesaw in the sand.

A man wearing a reflective vest and a hard hat holds a flag in the road

A person dressed in a blue coat is standing in on a busy sidewalk, studying painting of a street scene.

A man in green pants walking down the road.

The small child climbs on a red ropes on a playground.

You know i am looking like Justin Bieber.

A young man in a black and yellow jacket is gazing at something and smiling.

A man standing at a urinal with a coffee cup.

Five people walking with a multicolored sky in the background.

A old man having a beer alone.

A trained police dog sits next to his handler in front of the police van.

A person riding a bike on a snowy road.

Five men, uniformly dressed in white shirts, tie and black slacks converse at the back of an open van.

A man with a backwards hat works on machinery.

A black woman and a white man working in a factory setting packing jars with candles into boxes.

Asian man sweeping the walkway.

A man leans into a car to talk to the driver, as a man on a bicycle looks on.

Two young toddlers outside on the grass.

People are watching a person in a weird vehicle in a plaza.

A man walks by a silver vehicle.

A beautiful bride walking on a sidewalk with her new husband.

A little boy playing GameCube at a McDonald's.

A white dog shakes on the edge of a beach with an orange ball.

A group of people having a barbecue at a park.

A man in sunglasses puts his arm around a woman in a black and white blouse.

A man with a balloon hat and people eating outdoors at picnic tables.

A boy jump kicking over three kids kicking wood during a tae kwon do competition.

A boy in a red jacket pouring water on a man in a white shirt

A man with a red jacket is shielding himself from the sun trying to read a piece of paper.

Men walking down a street with children.

A little boy is standing on the street while a man in overalls is working on a stone wall.

A black dog leaps over a log.

A man in a suit is running past two other gentleman, also dressed in a suit.

Man in a red shirt riding his bicycle around water.

A barefooted man wearing olive green shorts grilling hotdogs on a small propane grill while holding a blue plastic cup.

A dog is running in the snow

A crowd is standing and waiting for the green light.

Man on skis looking at artwork for sale in the snow

Seven climbers are ascending a rock face whilst another man stands holding the rope.

The young gymnast's supple body soars above the balance beam.

A young boy is pushing a toy ATV around a rubber pool

Woman in red windbreaker looking though a rooftop binocu

----------------------------------------------Just for debugging--------------------------------------------------------

In [ ]:
with open(train_filepaths[1]) as myfile:
    firstNlines_en=myfile.readlines()[0:10]
with open(train_filepaths[0]) as myfile:
    firstNlines_de=myfile.readlines()[0:10]
data = []
for (raw_de, raw_en) in zip(firstNlines_de, firstNlines_en):
  de_tensor_ = torch.tensor([de_vocab[token] for token in de_tokenizer(raw_de)],
                          dtype=torch.long)
  en_tensor_ = torch.tensor([en_vocab[token] for token in en_tokenizer(raw_en)],
                          dtype=torch.long)
  data.append((de_tensor_, en_tensor_))
data,firstNlines_de, firstNlines_en

Out[33]: ([(tensor([ 22, 86, 258, 32, 88, 23, 95, 8, 17, 113, 7911, 3210,
 5, 4]),
 tensor([ 20, 26, 16, 1170, 809, 18, 58, 85, 337, 1340, 6, 5])),
 (tensor([ 85, 32, 11, 848, 2209, 16, 8269, 5, 4]),
 tensor([ 166, 37, 8, 336, 288, 18, 1225, 4, 759, 4497, 2958, 6,
 5])),
 (tensor([ 6, 70, 28, 220, 8, 16, 6770, 56, 509, 5, 4]),
 tensor([ 7, 62, 34, 233, 72, 4, 254, 4461, 6, 5])),
 (tensor([ 6, 13, 8, 7, 48, 42, 31, 12, 14, 544, 10, 699, 16, 249,
 5, 4]),
 tensor([ 7, 13, 8, 4, 31, 24, 11, 38, 10, 4, 590, 587, 4, 243,
 6, 5])),
 (tensor([ 22, 32, 54, 57, 1351, 10, 410, 175, 30, 5, 4]),
 tensor([ 20, 37, 18, 21, 9, 1204, 376, 135, 6, 5])),
 (tensor([ 6, 13, 8, 658, 40, 19, 134, 9, 38, 17, 106, 13,
 137, 42, 1780, 5, 4]),
 tensor([ 7, 13, 8, 52, 145, 4, 127, 30, 9, 76, 13, 1749,
 28, 24, 6, 5])),
 (tensor([ 6, 13, 173, 21, 5083, 4673, 24, 5, 4]),
 tensor([ 7, 13, 11, 137, 21, 4, 942, 2600, 5])),
 (tensor([ 6, 4260, 28, 187, 11, 27, 279, 38, 112, 4210,
 20, 37, 16554, 5, 4]),
 tensor([ 7, 6105, 34, 122, 10, 46, 296, 30, 3166, 4583, 42, 9,
 41, 6, 5])),
 (tensor([ 15, 18, 11, 14, 81, 1403, 62, 24, 7, 574, 120, 5,
 4]),
 tensor([ 7, 17, 15, 4, 63, 666, 11, 43, 49, 4, 1046, 6,
 5])),
 (tensor([ 93, 284, 339, 8, 17, 527, 12, 1209, 5, 4]),
 tensor([1780, 242, 10, 1062, 8, 9, 182, 14, 9, 313, 6, 5]))],
 ['Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n',
 'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.\n',
 'Ein kleines Mädchen klettert in ein Spielhaus aus Holz.\n',
 'Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.\n',
 'Zwei Männer stehen am Herd und bereiten Essen zu.\n',
 'Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.\n',
 'Ein Mann lächelt einen ausgestopften Löwen an.\n',
 'Ein schickes Mädchen spricht mit dem Handy während sie langsam die Straße entlangschwebt.\n',
 'Eine Frau mit einer großen Geldbörse geht an einem Tor vorbei.\n',
 'Jungen tanzen mitten in der Nacht auf Pfosten.\n'],
 ['Two young, White males are outside near many bushes.\n',
 'Several men in hard hats are operating a giant pulley system.\n',
 'A little girl climbing into a wooden playhouse.\n',
 'A man in a blue shirt is standing on a ladder cleaning a window.\n',
 'Two men are at the stove preparing food.\n',
 'A man in green holds a guitar while the other man observes his shirt.\n',
 'A man is smiling at a stuffed lion\n',
 'A trendy girl talking on her cellphone while gliding slowly down the street.\n',
 'A woman with a large purse is walking by a gate.\n',
 'Boys dancing on poles in the middle of the night.\n'])

In [ ]:
train_iter = DataLoader(data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)
for i in train_iter:
  print(i)

(tensor([[ 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [ 22, 6, 6, 93, 85, 6, 6, 15, 22, 6],
 [ 86, 13, 4260, 284, 32, 13, 70, 18, 32, 13],
 [ 258, 8, 28, 339, 11, 173, 28, 11, 54, 8],
 [ 32, 7, 187, 8, 848, 21, 220, 14, 57, 658],
 [ 88, 48, 11, 17, 2209, 5083, 8, 81, 1351, 40],
 [ 23, 42, 27, 527, 16, 4673, 16, 1403, 10, 19],
 [ 95, 31, 279, 12, 8269, 24, 6770, 62, 410, 134],
 [ 8, 12, 38, 1209, 5, 5, 56, 24, 175, 9],
 [ 17, 14, 112, 5, 4, 4, 509, 7, 30, 38],
 [ 113, 544, 4210, 4, 3, 3, 5, 574, 5, 17],
 [ 7911, 10, 20, 3, 1, 1, 4, 120, 4, 106],
 [ 3210, 699, 37, 1, 1, 1, 3, 5, 3, 13],
 [ 5, 16, 16554, 1, 1, 1, 1, 4, 1, 137],
 [ 4, 249, 5, 1, 1, 1, 1, 3, 1, 42],
 [ 3, 5, 4, 1, 1, 1, 1, 1, 1, 1780],
 [ 1, 4, 3, 1, 1, 1, 1, 1, 1, 5],
 [ 1, 3, 1, 1, 1, 1, 1, 1, 1, 4],
 [ 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]]), tensor([[ 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [ 20, 7, 7, 1780, 166, 7, 7, 7, 20, 7],
 [ 26, 13, 6105, 242, 37, 13, 62, 17, 37, 13],
 [ 16, 8, 34, 10, 8, 11, 34, 15, 18, 8],
 [1170, 4, 122, 1062, 336, 137, 233, 4, 21, 52],
 [ 809, 31, 10, 8, 288, 21, 72, 63, 9, 145],
 [ 18, 24, 46, 9, 18, 4, 4, 666, 1204, 4],
 [ 58, 11, 296, 182, 1225, 942, 254, 11, 376, 127],
 [ 85, 38, 30, 14, 4, 2600, 4461, 43, 135, 30],
 [ 337, 10, 3166, 9, 759, 5, 6, 49, 6, 9],
 [1340, 4, 4583, 313, 4497, 3, 5, 4, 5, 76],
 [ 6, 590, 42, 6, 2958, 1, 3, 1046, 3, 13],
 [ 5, 587, 9, 5, 6, 1, 1, 6, 1, 1749],
 [ 3, 4, 41, 3, 5, 1, 1, 5, 1, 28],
 [ 1, 243, 6, 1, 3, 1, 1, 3, 1, 24],
 [ 1, 6, 5, 1, 1, 1, 1, 1, 1, 6],
 [ 1, 5, 3, 1, 1, 1, 1, 1, 1, 5],
 [ 1, 3, 1, 1, 1, 1, 1, 1, 1, 3]]))

----------------------------------------------Debugging Ended--------------------------------------------------------

In [ ]:
train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_batch)

In [ ]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

***---------------------------------------------LSTM approach in Encoder Starting -----------------------------------------------------------------***

In [ ]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim: int, # Size of the one hot vectors that will be the input to the encoder
                 emb_dim: int, # Output size of the word embedding NN
                 hid_dim: int, 
                 n_layers: int, 
                 dropout: int):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim) # Shape --------------------> [input size, embedding dims]
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout) # Shape -----------> [embedding dims, hidden size, num layers]
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        # src shape : (sent_len, batch_size)
        
        embedded = self.dropout(self.embedding(src))
        # src embedded :(sent_len, batch_size, emb_dim)
        
        outputs, (hidden, cell) = self.rnn(embedded) 
        # Shape --> outputs  [Sequence_length , batch_size , hidden_size]
        # Shape --> (hs, cs) [num_layers, batch_size size, hidden_size]
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers        
        self.embedding = nn.Embedding(output_dim, emb_dim) #Shape --------------------> [output_dim size, embedding dims]
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout) # Shape ----------->  [embedding dims, hidden size, num layers]
        self.fc_out = nn.Linear(hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
       
        input = input.unsqueeze(0) # shape of input = batch_size but we want (1, batch_size)
        embedded = self.dropout(self.embedding(input)) # Shape -----------> [1, batch_size, embedding dims]
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        # Shape --> outputs [1, batch_size , hidden_size]
        # Shape --> (hs, cs) [num_layers, batch_size size, hidden_size] (passing encoder's hs, cs - context vectors)
    
    
        prediction = self.fc_out(output.squeeze(0))
        # Shape --> predictions [ batch_size , output_size]
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
       
        # Shape - src : (10, 32) [(Sentence length German + some padding), Number of Sentences]
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device) # Shape --> outputs (14, 32, 5766) [(Sentence length German + some padding), Number of Sentences, Vocab_size]
        
        hidden, cell = self.encoder(src)
        input = trg[0,:] # Grab start Token(Trigger token <SOS>)
        # Shape of input (32 elements)
        
        for t in range(1, trg_len):
            output, hidden, cell = self.decoder(input, hidden, cell) # For the first iteration 
            # Shape --> output (batch_size, Vocab length) 
            outputs[t] = output # 0th dimension is batch size, 1st dimension is word embedding
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1) 
            input = trg[t] if teacher_force else top1        
        return outputs

In [ ]:
INPUT_DIM = len(de_vocab.itos)
OUTPUT_DIM = len(en_vocab.itos)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Out[23]: Seq2Seq(
 (encoder): Encoder(
 (embedding): Embedding(19215, 256)
 (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
 (dropout): Dropout(p=0.5, inplace=False)
 )
 (decoder): Decoder(
 (embedding): Embedding(10838, 256)
 (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
 (fc_out): Linear(in_features=512, out_features=10838, bias=True)
 (dropout): Dropout(p=0.5, inplace=False)
 )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 20,609,878 trainable parameters

In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = PAD_IDX
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()    
    epoch_loss = 0
    for i, (src,trg) in enumerate(iterator):
        optimizer.zero_grad()
        output = model(src, trg)      
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)       
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, (src,trg) in enumerate(iterator):
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')
for epoch in range(N_EPOCHS):  
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 11m 53s
	Train Loss: 4.999 | Train PPL: 148.268
 Val. Loss: 5.143 | Val. PPL: 171.247
Epoch: 02 | Time: 11m 31s
	Train Loss: 4.381 | Train PPL: 79.915
 Val. Loss: 4.969 | Val. PPL: 143.865
Epoch: 03 | Time: 11m 30s
	Train Loss: 4.085 | Train PPL: 59.436
 Val. Loss: 4.687 | Val. PPL: 108.537
Epoch: 04 | Time: 12m 33s
	Train Loss: 3.830 | Train PPL: 46.051
 Val. Loss: 4.481 | Val. PPL: 88.336
Epoch: 05 | Time: 11m 58s
	Train Loss: 3.651 | Train PPL: 38.505
 Val. Loss: 4.441 | Val. PPL: 84.843
Epoch: 06 | Time: 11m 56s
	Train Loss: 3.513 | Train PPL: 33.536
 Val. Loss: 4.218 | Val. PPL: 67.931
Epoch: 07 | Time: 11m 49s
	Train Loss: 3.362 | Train PPL: 28.850
 Val. Loss: 4.264 | Val. PPL: 71.080
Epoch: 08 | Time: 12m 17s
	Train Loss: 3.212 | Train PPL: 24.820
 Val. Loss: 4.135 | Val. PPL: 62.475
Epoch: 09 | Time: 12m 4s
	Train Loss: 3.071 | Train PPL: 21.562
 Val. Loss: 4.166 | Val. PPL: 64.466
Epoch: 10 | Time: 11m 52s
	Train Loss: 2.928 | Train PPL: 18.688
 Val. Loss: 4.072 | Val. PPL: 58.699

In [ ]:
model.parameters

Out[31]: <bound method Module.parameters of Seq2Seq(
 (encoder): Encoder(
 (embedding): Embedding(19215, 256)
 (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
 (dropout): Dropout(p=0.5, inplace=False)
 )
 (decoder): Decoder(
 (embedding): Embedding(10838, 256)
 (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
 (fc_out): Linear(in_features=512, out_features=10838, bias=True)
 (dropout): Dropout(p=0.5, inplace=False)
 )
)>

In [ ]:
def translate_sentence(model, sentence, german, english, device, max_length=50):
  de_tokenizer = get_tokenizer('spacy', language='de')
  en_tokenizer = get_tokenizer('spacy', language='en')
  tokens = [token for token in de_tokenizer(sentence)]
  tokens.insert(0, '<bos>')
  tokens.append('<eos>')
  text_to_indices = [german.stoi[token] for token in tokens]
  sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)
  with torch.no_grad():
    hidden, cell = model.encoder(sentence_tensor)
  outputs = [en_vocab.stoi["<sos>"]]
  for _ in range(50):
        previous_word = torch.LongTensor([outputs[-1]]).to(device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)
            best_guess = output.argmax(1).item()
        outputs.append(best_guess)
        if output.argmax(1).item() == english.stoi["<eos>"]:
            break
  translated_sentence = [en_vocab.itos[idx] for idx in outputs]            
  return translated_sentence

In [ ]:
sentence = "ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster."
translate_sentence(model, sentence, de_vocab, en_vocab, device)

Out[118]: ['<unk>',
 'Man',
 'in',
 'a',
 'blue',
 'shirt',
 'is',
 'on',
 'a',
 'bench',
 'on',
 'a',
 'sidewalk',
 '.',
 '\n',
 '<eos>']

***---------------------------------------------LSTM approach in Encoder Ending -----------------------------------------------------------------***

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: float):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.dropout = dropout

        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)
        
        
    def forward(self,
                src: Tensor) -> Tuple[Tensor]:

        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))

        return outputs, hidden

In [ ]:
class Attention(nn.Module):
    def __init__(self,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 attn_dim: int):
        super().__init__()

        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim

        self.attn_in = (enc_hid_dim * 2) + dec_hid_dim

        self.attn = nn.Linear(self.attn_in, attn_dim)

    def forward(self,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tensor:

        src_len = encoder_outputs.shape[0]

        repeated_decoder_hidden = decoder_hidden.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        energy = torch.tanh(self.attn(torch.cat((
            repeated_decoder_hidden,
            encoder_outputs),
            dim = 2)))

        attention = torch.sum(energy, dim=2)

        return F.softmax(attention, dim=1)

In [ ]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 dec_hid_dim: int,
                 dropout: int,
                 attention: nn.Module):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dec_hid_dim = dec_hid_dim
        self.output_dim = output_dim
        self.dropout = dropout
        self.attention = attention

        self.embedding = nn.Embedding(output_dim, emb_dim)

        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)

        self.out = nn.Linear(self.attention.attn_in + emb_dim, output_dim)

        self.dropout = nn.Dropout(dropout)


    def _weighted_encoder_rep(self,
                              decoder_hidden: Tensor,
                              encoder_outputs: Tensor) -> Tensor:

        a = self.attention(decoder_hidden, encoder_outputs)

        a = a.unsqueeze(1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)

        weighted_encoder_rep = torch.bmm(a, encoder_outputs)

        weighted_encoder_rep = weighted_encoder_rep.permute(1, 0, 2)

        return weighted_encoder_rep


    def forward(self,
                input: Tensor,
                decoder_hidden: Tensor,
                encoder_outputs: Tensor) -> Tuple[Tensor]:

        input = input.unsqueeze(0)

        embedded = self.dropout(self.embedding(input))

        weighted_encoder_rep = self._weighted_encoder_rep(decoder_hidden,
                                                          encoder_outputs)

        rnn_input = torch.cat((embedded, weighted_encoder_rep), dim = 2)

        output, decoder_hidden = self.rnn(rnn_input, decoder_hidden.unsqueeze(0))

        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted_encoder_rep = weighted_encoder_rep.squeeze(0)

        output = self.out(torch.cat((output,
                                     weighted_encoder_rep,
                                     embedded), dim = 1))

        return output, decoder_hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden = self.decoder(output, hidden, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs

In [ ]:
INPUT_DIM = len(de_vocab)
OUTPUT_DIM = len(en_vocab)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 32
DEC_EMB_DIM = 32
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)

attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

model = Seq2Seq(enc, dec, device).to(device)

In [ ]:
def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,491,070 trainable parameters

In [ ]:
PAD_IDX = en_vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)

            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

test_loss = evaluate(model, test_iter, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 5m 31s
	Train Loss: 5.747 | Train PPL: 313.281
 Val. Loss: 5.185 | Val. PPL: 178.501
Epoch: 02 | Time: 5m 51s
	Train Loss: 4.734 | Train PPL: 113.712
 Val. Loss: 5.050 | Val. PPL: 155.997
Epoch: 03 | Time: 5m 38s
	Train Loss: 4.507 | Train PPL: 90.676
 Val. Loss: 4.946 | Val. PPL: 140.666
Epoch: 04 | Time: 5m 32s
	Train Loss: 4.362 | Train PPL: 78.426
 Val. Loss: 4.818 | Val. PPL: 123.725
Epoch: 05 | Time: 5m 42s
	Train Loss: 4.234 | Train PPL: 69.003
 Val. Loss: 4.763 | Val. PPL: 117.142
Epoch: 06 | Time: 5m 33s
	Train Loss: 4.132 | Train PPL: 62.329
 Val. Loss: 4.679 | Val. PPL: 107.676
Epoch: 07 | Time: 5m 49s
	Train Loss: 4.025 | Train PPL: 55.987
 Val. Loss: 4.643 | Val. PPL: 103.856
Epoch: 08 | Time: 5m 35s
	Train Loss: 3.953 | Train PPL: 52.073
 Val. Loss: 4.582 | Val. PPL: 97.671
Epoch: 09 | Time: 5m 43s
	Train Loss: 3.898 | Train PPL: 49.299
 Val. Loss: 4.513 | Val. PPL: 91.223
Epoch: 10 | Time: 6m 2s
	Train Loss: 3.810 | Train PPL: 45.167
 Val. Loss: 4.502 | Val. PPL: 90.214
 Test Loss: 4.476 | Test PPL: 87.898 |

In [ ]:
model.parameters

Out[83]: <bound method Module.parameters of Seq2Seq(
 (encoder): Encoder(
 (embedding): Embedding(19215, 32)
 (rnn): GRU(32, 64, bidirectional=True)
 (fc): Linear(in_features=128, out_features=64, bias=True)
 (dropout): Dropout(p=0.5, inplace=False)
 )
 (decoder): Decoder(
 (attention): Attention(
 (attn): Linear(in_features=192, out_features=8, bias=True)
 )
 (embedding): Embedding(10838, 32)
 (rnn): GRU(160, 64)
 (out): Linear(in_features=224, out_features=10838, bias=True)
 (dropout): Dropout(p=0.5, inplace=False)
 )
)>